# Libs and Inputs

In [54]:
#Environment PyMatlab
from __future__ import division
import numpy as np
import time
from tqdm import tqdm

MaxLCOE_Kite= 200 # $/MWh, Max LCOE for a kite location to be considered by the optimization model
MaxLCOE_Wind= 300 # $/MWh, Max LCOE for a kite location to be considered by the optimization model

MaxPowerTransmissionWind=600 #MW Maximum input power to the transmission system
MaxPowerTransmissionKite=60 #MW Maximum input power to the transmission system
ResultsFileName='PortfolioOptimizationWindWaveOcean'

ShapeFileCoast="./GEO_data/ne_10m_coastline.shp"
ShapeFileStates="./GEO_data/ne_10m_admin_1_states_provinces_lines.shp"
WindNpzPath='../Wind_AE/WindEnergyNREL_100m_Haliade150_6MW.npz'
KiteLFGenerationNpzPath ='../KiteLF_Optimization/LF_KiteTimeSeriesGeneration.npz'
TransmissionNpzPathWind ='../Transmission/Transmission_'+str(MaxPowerTransmissionWind)+'MW.npz'
TransmissionNpzPathKite ='../Transmission/Transmission_'+str(MaxPowerTransmissionKite)+'MW.npz'

# Kite

In [50]:
KiteLFGenerationNpz=np.load(KiteLFGenerationNpzPath, allow_pickle=True)
lat=KiteLFGenerationNpz["lat"][0]
long=KiteLFGenerationNpz["long"][0]
AnnualizedCostKite=[]
KiteLatLong=[] #[#Kite Sites,2]
KiteEnergy =[] #[#Kite Sites, Length of Time Series] units (MW)
MaxNumKitesPerSite=[] #[#Kite Sites] Maximum Number of Kites that can be placed at each site (Default 100)
CLCDs=[]
LCOEs_kite=[]
theta_vecs=[]
l_vecs=[]
uopt_vecs=[]
X_Y_Vecs=[]
mFuse_vec=[]
mWing_vec=[]
Power_LF_AVG=[]


KiteResults=KiteLFGenerationNpz["Results"]

for i in range(len(KiteResults)):
    KiteData=KiteResults[i]
    X=KiteData["X"]
    Y=KiteData["Y"]
    Latitude=lat[X-1]
    Longitude=long[Y-1]
    uopt=np.asarray(KiteData["uopt"])[0]
    mFuse=KiteData["mFuse"]
    mWing=KiteData["mWing"]
    LCOE=KiteData["LCOEBest"]*1000 # $/Mwh
    CLCD     =np.asarray(KiteData["CLCD"])[0]
    Jopt_vec =np.asarray(KiteData["Jopt_vec"])[0] #Kwh
    theta_vec=np.asarray(KiteData["theta_vec"])[0]
    l_vec    =np.asarray(KiteData["l_vec"])[0]
    Power=KiteData["Power"]/1000

    if LCOE<=MaxLCOE_Kite:
        KiteLatLong.append([Latitude,Longitude])
        KiteEnergy.append(Jopt_vec/1000) #MW
        MaxNumKitesPerSite.append(100)
        CLCDs.append(CLCD)
        LCOEs_kite.append(LCOE)
        theta_vecs.append(theta_vec)
        l_vecs.append(l_vec)
        uopt_vecs.append(uopt)
        X_Y_Vecs.append([X,Y])
        mFuse_vec.append(mFuse)
        mWing_vec.append(mWing)
        Power_LF_AVG.append(KiteData["Power"]/1000)

        #Reconstruct the annualized cost
        AverageGen=KiteData["Power"]/1000
        AnnualizedCostKite.append(LCOE*8760*AverageGen) # $/year

# Wind

In [51]:
WindNpz=np.load(WindNpzPath, allow_pickle=True)
WindEnergy=WindNpz["WindEnergy"]*6 #MW
WindLatLong=WindNpz["LatLong"]
AnnualizedCostWind=WindNpz["AnnualizedCostWind"]*10**6 # $/Year
WindEnergy_Adjusted=np.zeros((WindEnergy.shape[0],87))


#sample wind energy such that it has 87 time steps as in the kite data (default until we fix the kite data)
for i in range(87):
    WindEnergy_Adjusted[:,i]=np.average(WindEnergy[:,4*i:4*(i+1)])

MaxNumWindPerSite=[4]*WindEnergy_Adjusted.shape[0]
LCOE_Wind=AnnualizedCostWind/(np.average(WindEnergy_Adjusted,axis=1)*8760) # $/MWh

# Transmission

In [52]:
Wind_TransmissionNpz=np.load(TransmissionNpzPathWind, allow_pickle=True)
Wind_TransLatLong=Wind_TransmissionNpz["LatLong"]
Wind_EfficiencyTransmission=Wind_TransmissionNpz["S_Efficiency"]
Wind_AnnualizedCostTransmission=Wind_TransmissionNpz["S_BestACost"]*10**6 #M$/Year to $/Year

Kite_TransmissionNpz=np.load(TransmissionNpzPathKite, allow_pickle=True)
Kite_TransLatLong=Kite_TransmissionNpz["LatLong"]
Kite_EfficiencyTransmission=Kite_TransmissionNpz["S_Efficiency"]
Kite_AnnualizedCostTransmission=Kite_TransmissionNpz["S_BestACost"]*10**6 #M$/Year to $/Year



# Save In a Single NPZ

In [53]:
np.savez("PreprocessedData.npz",WindEnergy=WindEnergy_Adjusted,WindLatLong=WindLatLong, AnnualizedCostWind=AnnualizedCostWind,
         LCOE_Wind=LCOE_Wind,MaxNumWindPerSite=MaxNumWindPerSite,
         KiteEnergy=KiteEnergy,KiteLatLong=KiteLatLong, AnnualizedCostKite=AnnualizedCostKite,
         LCOEs_kite=LCOEs_kite,MaxNumKitesPerSite=MaxNumKitesPerSite, CLCDs=CLCDs,theta_vecs=theta_vecs,l_vecs=l_vecs,
         uopt_vecs=uopt_vecs,X_Y_Vecs=X_Y_Vecs,mFuse_vec=mFuse_vec,mWing_vec=mWing_vec,Power_LF_AVG=Power_LF_AVG,
         Wind_AnnualizedCostTransmission=Wind_AnnualizedCostTransmission,Wind_TransLatLong=Wind_TransLatLong,Wind_EfficiencyTransmission=Wind_EfficiencyTransmission,
         Kite_TransLatLong=Kite_TransLatLong, Kite_EfficiencyTransmission=Kite_EfficiencyTransmission, Kite_AnnualizedCostTransmission=Kite_AnnualizedCostTransmission,
         Wind_MaxPowerTransmission=MaxPowerTransmissionWind,Kite_MaxPowerTransmission=MaxPowerTransmissionKite,
         TimeStepHours=4.19)
